<a href="https://colab.research.google.com/github/bdubreu/fastpractice/blob/master/BatchNorm_vs_EvoNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torchvision.datasets import MNIST
import torch.nn.functional as F

In [3]:
trainset = MNIST('../', download=True, train=True)
testset = MNIST('../', download=True, train=False)

y_trainset = trainset.targets
y_testset = testset.targets

# this time, we resize the data to have directly one channel for convolutions
trainset = trainset.data.reshape(60000, 1, 28, 28).to(torch.float32)
testset = testset.data.reshape(10000, 1, 28, 28).to(torch.float32)

# normalize
m, s = trainset.mean(), trainset.std()
trainset = (trainset - m) / s
testset = (testset - m) / s

Extracting ../MNIST/raw/train-images-idx3-ubyte.gz to ../MNIST/raw


Extracting ../MNIST/raw/train-labels-idx1-ubyte.gz to ../MNIST/raw


Extracting ../MNIST/raw/t10k-images-idx3-ubyte.gz to ../MNIST/raw


Extracting ../MNIST/raw/t10k-labels-idx1-ubyte.gz to ../MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

from torch.optim import SGD

In [0]:
def accuracy(output, target):
    return (torch.argmax(output, dim=1) == target).float().mean()

In [6]:
class MNIST_Dataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

    def __len__(self):
        return len(self.x)

In [0]:
x_train, x_valid = trainset[0:50000, :], trainset[50000:, :]
y_train, y_valid = y_trainset[0:50000], y_trainset[50000:]

train = MNIST_Dataset(x_train, y_train)
valid = MNIST_Dataset(x_valid, y_valid)

In [0]:
EPOCHS = 5
bs = 64
lr = 0.05
loss_func = F.cross_entropy


train_dl = DataLoader(train, bs, shuffle=True)
valid_dl = DataLoader(valid, bs, shuffle=False)

In [0]:
# Define model
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


def flatten(x):
    return x.view(x.shape[0], -1)


model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=4,
              kernel_size=3, stride=2, padding=1),  # bs*4*14*14
    nn.ReLU(),
    nn.Conv2d(4, 8, 3, 2, 1),  # bs * 8 * 7 * 7
    nn.ReLU(),
    nn.Conv2d(8, 16, 3, 2, 1),  # bs * 16 * 4 * 4
    nn.ReLU(),
    nn.Conv2d(16, 32, 3, 2, 1),  # bs * 32 * 2 * 2
    nn.ReLU(),
    nn.Conv2d(32, 64, 3, 2, 1),  # bs * 64 * 1 * 1
    nn.AdaptiveAvgPool2d(1),
    Lambda(flatten),
    nn.Linear(64, 10)
)

opt = SGD(model.parameters(), lr)

In [10]:
for epoch in range(EPOCHS):
    model.train()
    for xb, yb in train_dl:
        loss = loss_func(model.cuda()(xb.cuda()), yb.cuda())
        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        total_loss, total_acc = 0., 0.
        for xb, yb in valid_dl:
            pred = model(xb.cuda())
            total_loss += loss_func(pred, yb.cuda())
            total_acc += accuracy(pred, yb.cuda())
        n_entries = len(valid_dl)
        print('epoch', epoch,
              'loss:', (total_loss/n_entries).item(),
              'accuracy:', (total_acc/n_entries).item()
              )

epoch 0 loss: 0.892424464225769 accuracy: 0.7540804147720337
epoch 1 loss: 0.1958770602941513 accuracy: 0.9364052414894104
epoch 2 loss: 0.19175830483436584 accuracy: 0.9442675113677979
epoch 3 loss: 0.17649568617343903 accuracy: 0.9483479261398315
epoch 4 loss: 0.1715310513973236 accuracy: 0.950039803981781


# Add BatchNorm

In [0]:
# This is starting to be messy. We'll see if we can factor this out
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=8,
              kernel_size=3, stride=2, padding=1, bias=False),  # bs*4*14*14
    nn.BatchNorm2d(8),
    nn.ReLU(),
    nn.Conv2d(8, 16, 3, 2, 1, bias=False),  # bs * 8 * 7 * 7
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.Conv2d(16, 32, 3, 2, 1, bias=False),  # bs * 16 * 4 * 4
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32, 64, 3, 2, 1, bias=False),  # bs * 32 * 2 * 2
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3, 2, 1, bias=False),  # bs * 64 * 1 * 1
    nn.BatchNorm2d(64),
    nn.AdaptiveAvgPool2d(1),
    Lambda(flatten),
    nn.Linear(64, 10)
)

In [0]:
opt = SGD(model.parameters(), lr)

In [13]:
for epoch in range(EPOCHS):
    model.train()
    for xb, yb in train_dl:
        loss = loss_func(model.cuda()(xb.cuda()), yb.cuda())
        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        total_loss, total_acc = 0., 0.
        for xb, yb in valid_dl:
            pred = model(xb.cuda())
            total_loss += loss_func(pred, yb.cuda())
            total_acc += accuracy(pred, yb.cuda())
        n_entries = len(valid_dl)
        print('epoch', epoch,
              'loss:', (total_loss/n_entries).item(),
              'accuracy:', (total_acc/n_entries).item()
              )

epoch 0 loss: 0.08148886263370514 accuracy: 0.9772093892097473
epoch 1 loss: 0.054623913019895554 accuracy: 0.9842755198478699
epoch 2 loss: 0.5990749597549438 accuracy: 0.8552945852279663
epoch 3 loss: 0.0878567323088646 accuracy: 0.9727309346199036
epoch 4 loss: 0.04501421004533768 accuracy: 0.9870620965957642


# Try EVONorm

In [0]:
import torch
import torch.nn as nn


class SwishImplementation(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        ctx.save_for_backward(i)
        return i * torch.sigmoid(i)

    @staticmethod
    def backward(ctx, grad_output):
        sigmoid_i = torch.sigmoid(ctx.saved_variables[0])
        return grad_output * (sigmoid_i * (1 + ctx.saved_variables[0] * (1 - sigmoid_i)))


class MemoryEfficientSwish(nn.Module):
    def forward(self, x):
        return SwishImplementation.apply(x)

def instance_std(x, eps=1e-5):
    var = torch.var(x, dim = (2, 3), keepdim=True).expand_as(x)
    if torch.isnan(var).any():
        var = torch.zeros(var.shape)
    return torch.sqrt(var + eps)

def group_std(x, groups = 32, eps = 1e-5):
    N, C, H, W = x.size()
    x = torch.reshape(x, (N, groups, C // groups, H, W))
    var = torch.var(x, dim = (2, 3, 4), keepdim = True).expand_as(x)
    return torch.reshape(torch.sqrt(var + eps), (N, C, H, W))

class EvoNorm2D(nn.Module):

    def __init__(self, input, non_linear=True, version='B0', efficient=False, affine=True, momentum=0.9, eps=1e-5, groups=32, training=True):
        super(EvoNorm2D, self).__init__()
        self.non_linear = non_linear
        self.version = version
        self.training = training
        self.momentum = momentum
        self.efficient = efficient
        if self.version == 'S0':
            self.swish = MemoryEfficientSwish()
        self.groups = groups
        self.eps = eps
        if self.version not in ['B0', 'S0']:
            raise ValueError("Invalid EvoNorm version")
        self.insize = input
        self.affine = affine

        if self.affine:
            self.gamma = nn.Parameter(torch.ones(1, self.insize, 1, 1))
            self.beta = nn.Parameter(torch.zeros(1, self.insize, 1, 1))
            if self.non_linear:
                self.v = nn.Parameter(torch.ones(1,self.insize,1,1))
        else:
            self.register_parameter('gamma', None)
            self.register_parameter('beta', None)
            self.register_buffer('v', None)
        self.register_buffer('running_var', torch.ones(1, self.insize, 1, 1))

        self.reset_parameters()

    def reset_parameters(self):
        self.running_var.fill_(1)

    def _check_input_dim(self, x):
        if x.dim() != 4:
            raise ValueError('expected 4D input (got {}D input)'
                             .format(x.dim()))
    
    def forward(self, x):
        self._check_input_dim(x)
        if self.version == 'S0':
            if self.non_linear:
                if not self.efficient:
                    num = x * torch.sigmoid(self.v * x)   # Original Swish Implementation, however memory intensive.
                else:
                    num = self.swish(x)    # Experimental Memory Efficient Variant of Swish
                return num / group_std(x, groups = self.groups, eps = self.eps) * self.gamma + self.beta
            else:
                return x * self.gamma + self.beta
        if self.version == 'B0':
            if self.training:
                var = torch.var(x, dim=(0, 2, 3), unbiased=False, keepdim=True)
                self.running_var.mul_(self.momentum)
                self.running_var.add_((1 - self.momentum) * var)
                #print('yo')
            else:
                var = self.running_var

            if self.non_linear:
                den = torch.max((var+self.eps).sqrt(), self.v * x + instance_std(x, eps=self.eps).cuda())
                return x / den * self.gamma + self.beta
            else:
                return x * self.gamma + self.beta

In [0]:
# This is starting to be messy. We'll see if we can factor this out
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=8,
              kernel_size=3, stride=2, padding=1, bias=False),  # bs*4*14*14
    EvoNorm2D(8),
    #nn.ReLU(),
    nn.Conv2d(8, 16, 3, 2, 1, bias=False),  # bs * 8 * 7 * 7
    EvoNorm2D(16),
    #nn.ReLU(),
    nn.Conv2d(16, 32, 3, 2, 1, bias=False),  # bs * 16 * 4 * 4
    EvoNorm2D(32),
    #nn.ReLU(),
    nn.Conv2d(32, 64, 3, 2, 1, bias=False),  # bs * 32 * 2 * 2
    EvoNorm2D(64),
    #nn.ReLU(),
    nn.Conv2d(64, 64, 3, 2, 1, bias=False),  # bs * 64 * 1 * 1
    EvoNorm2D(64),
    nn.AdaptiveAvgPool2d(1),
    Lambda(flatten),
    nn.Linear(64, 10)
)

In [16]:
opt = SGD(model.parameters(), lr)
for epoch in range(EPOCHS):
    model.train()
    for xb, yb in train_dl:
        loss = loss_func(model.cuda()(xb.cuda()), yb.cuda())
        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        total_loss, total_acc = 0., 0.
        for xb, yb in valid_dl:
            pred = model(xb.cuda())
            total_loss += loss_func(pred, yb.cuda())
            total_acc += accuracy(pred, yb.cuda())
        n_entries = len(valid_dl)
        print('epoch', epoch,
              'loss:', (total_loss/n_entries).item(),
              'accuracy:', (total_acc/n_entries).item()
              )

epoch 0 loss: 0.10983648151159286 accuracy: 0.9673566818237305
epoch 1 loss: 0.06782960891723633 accuracy: 0.981090784072876
epoch 2 loss: 0.3812379240989685 accuracy: 0.9166003465652466
epoch 3 loss: 0.051979053765535355 accuracy: 0.984175980091095
epoch 4 loss: 0.0521659180521965 accuracy: 0.9856687784194946
